<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Capacitated Facility Location - Problem #01



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>


<a id='prob'></a>
## Problem #02

<br>
Facilities <br>
$|I| = 16$ 
<br>
<br>
Clients <br>
$|J| = 50$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcap71.txt'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))


start = time.time()
# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Using license file /Users/danielcinalli/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-27
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 850 rows, 816 columns and 2400 nonzeros
Model fingerprint: 0xab630171
Variable types: 800 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2063082.9875
Presolve removed 813 rows and 782 columns
Presolve time: 0.07s
Presolved: 37 rows, 34 columns, 81 nonzeros
Found heuristic solution: objective 937096.30000
Variable types: 0 continuous, 34 integer (34 binary)

Root relaxation: objective 9.326158e+05, 9 iterations, 0.01 seconds

    Nodes    |    Current Node    

In [14]:
#num_facilities
#num_customers
#capacity
#fixed_cost 
#cartesian_prod
#demand
#alloc_cost
shipping_cost


{(0, 0): 6739.725,
 (0, 1): 10355.05,
 (0, 2): 7650.4,
 (0, 3): 5219.5,
 (0, 4): 5776.125,
 (0, 5): 6641.175,
 (0, 6): 4374.525,
 (0, 7): 3847.1,
 (0, 8): 6429.475,
 (0, 9): 5396.525,
 (0, 10): 5219.5,
 (0, 11): 4182.9,
 (0, 12): 7391.25,
 (0, 13): 5038.825,
 (0, 14): 10349.575,
 (0, 15): 6051.7,
 (1, 0): 3204.8625,
 (1, 1): 5457.075,
 (1, 2): 3845.4,
 (1, 3): 2396.85,
 (1, 4): 2628.4875,
 (1, 5): 3220.0875,
 (1, 6): 1838.9625,
 (1, 7): 2266.35,
 (1, 8): 3117.8625,
 (1, 9): 2582.8125,
 (1, 10): 2296.8,
 (1, 11): 1779.15,
 (1, 12): 5115.6,
 (1, 13): 2189.1375,
 (1, 14): 5399.4375,
 (1, 15): 2838.375,
 (2, 0): 4914.0,
 (2, 1): 26409.6,
 (2, 2): 19622.4,
 (2, 3): 13876.8,
 (2, 4): 9147.6,
 (2, 5): 14977.2,
 (2, 6): 21848.4,
 (2, 7): 35330.4,
 (2, 8): 15111.6,
 (2, 9): 23679.6,
 (2, 10): 9828.0,
 (2, 11): 19303.2,
 (2, 12): 57472.8,
 (2, 13): 11180.4,
 (2, 14): 22957.2,
 (2, 15): 15489.6,
 (3, 0): 32372.1125,
 (3, 1): 29982.225,
 (3, 2): 21024.325,
 (3, 3): 29681.4,
 (3, 4): 21275.0125,
 (

In [2]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_02_Simplex.lp')


Build a warehouse at location 1.

Build a warehouse at location 2.

Build a warehouse at location 3.

Build a warehouse at location 4.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 11.

Build a warehouse at location 12.

Build a warehouse at location 13.

Client 1 receives 100.0 % of its demand  from Warehouse 8 .

Client 2 receives 100.0 % of its demand  from Warehouse 12 .

Client 3 receives 100.0 % of its demand  from Warehouse 1 .

Client 4 receives 100.0 % of its demand  from Warehouse 6 .

Client 5 receives 100.0 % of its demand  from Warehouse 8 .

Client 6 receives 100.0 % of its demand  from Warehouse 1 .

Client 7 receives 100.0 % of its demand  from Warehouse 2 .

Client 8 receives 100.0 % of its demand  from Warehouse 3 .

Client 9 receives 100.0 % of its demand  from Warehouse 8 .

Client 10 receives 100.0 % of its demand  from Warehouse 8 .

Client 11 

<br>
<br>

<a id='random'></a>
### Random Heuristic 



In [3]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

start = time.time() 
result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)


end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
2856902.3
TIME IS:  0.004380941390991211


In [4]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 2.

Build a warehouse at location 3.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 11.

Build a warehouse at location 12.

Build a warehouse at location 13.

Build a warehouse at location 14.

Build a warehouse at location 15.

Build a warehouse at location 16.

Client 1 receives 100% of its demand  from Warehouse 15 .

Client 2 receives 100% of its demand  from Warehouse 10 .

Client 3 receives 100% of its demand  from Warehouse 10 .

Client 4 receives 100% of its demand  from Warehouse 2 .

Client 5 receives 100% of its demand  from Warehouse 5 .

Client 6 receives 100% of its demand  from Warehouse 11 .

Client 7 receives 100% of its demand  from Warehouse 13 .

Client 8 receives 100% of its demand 

In [5]:
len(facs)

16

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [6]:
#Get the lowest shipping cost for each Client

start = time.time()
#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

{(0, 7): 3847.1, (1, 11): 1779.15, (2, 0): 4914.0, (3, 5): 20071.7125, (4, 7): 955.575, (5, 0): 6421.5125, (6, 1): 28499.25, (7, 2): 6370.65, (8, 7): 1211.925, (9, 7): 546.4, (10, 3): 12638.5, (11, 4): 1231.7, (12, 5): 5185.975, (13, 0): 1953.7375, (14, 6): 7310.8125, (15, 7): 8135.7, (16, 3): 2542.5, (17, 8): 11423.1, (18, 3): 3542.0, (19, 6): 5467.3125, (20, 3): 693.5, (21, 9): 11166.8625, (22, 10): 0.0, (23, 0): 4883.0, (24, 11): 5270.65, (25, 4): 2952.9625, (26, 12): 124051.2, (27, 10): 1601.175, (28, 13): 2958.275, (29, 14): 8049.9375, (30, 15): 3285.975, (31, 10): 2398.9, (32, 15): 7346.625, (33, 2): 203364.0, (34, 11): 4891.25, (35, 11): 4154.1, (36, 5): 56946.3875, (37, 5): 15076.0625, (38, 7): 34351.125, (39, 4): 5211.1, (40, 10): 10086.0, (41, 3): 13627.4, (42, 7): 7885.625, (43, 6): 9331.25, (44, 12): 89976.15, (45, 7): 39050.575, (46, 7): 10134.3, (47, 6): 1756.0375, (48, 5): 20807.1, (49, 11): 2614.05}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
112500
950470.18

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [7]:
#Get the greatest shipping cost for each Client

start = time.time()
#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

{(0, 1): 10355.05, (1, 1): 5457.075, (2, 12): 57472.8, (3, 12): 135170.7, (4, 14): 2512.9375, (5, 12): 45474.65, (6, 12): 259515.0, (7, 12): 105796.35, (8, 14): 2869.7625, (9, 14): 2314.8, (10, 12): 461992.125, (11, 12): 79235.6, (12, 12): 135275.15, (13, 12): 13134.55, (14, 12): 39913.5, (15, 14): 39867.75, (16, 12): 17458.5, (17, 12): 274079.0, (18, 12): 21789.625, (19, 1): 14113.125, (20, 12): 3066.6, (21, 12): 56025.975, (22, 12): 46945.2, (23, 12): 28226.4, (24, 12): 59259.2, (25, 12): 29479.075, (26, 1): 406770.0, (27, 12): 49650.85, (28, 12): 37089.9, (29, 12): 50490.0, (30, 12): 18722.55, (31, 12): 26701.85, (32, 12): 53121.75, (33, 12): 1361570.4, (34, 12): 21336.25, (35, 12): 26946.75, (36, 12): 378755.425, (37, 12): 209073.175, (38, 14): 73399.3125, (39, 12): 32644.2, (40, 12): 133135.2, (41, 12): 91957.025, (42, 14): 23103.4375, (43, 1): 32575.0, (44, 14): 224408.1375, (45, 14): 75966.2875, (46, 14): 21287.025, (47, 1): 3928.575, (48, 12): 147937.2, (49, 12): 14979.45}
[1, 

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [8]:
#Get the lowest shipping cost for each Client


start = time.time()
#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

{(0, 10): 5219.5, (1, 10): 2296.8, (2, 10): 9828.0, (3, 10): 23096.675, (4, 10): 1133.05, (5, 10): 7029.425, (6, 1): 35999.25, (7, 10): 12741.3, (8, 10): 1395.9, (9, 10): 928.8, (10, 3): 20138.5, (11, 10): 2463.4, (12, 10): 10371.95, (13, 10): 2434.575, (14, 10): 14621.625, (15, 7): 15635.7, (16, 10): 3062.3, (17, 8): 18923.1, (18, 10): 4123.9, (19, 10): 7312.5, (20, 10): 780.9, (21, 9): 18666.8625, (22, 10): 0.0, (23, 10): 7106.0, (24, 10): 10541.3, (25, 10): 3268.9, (26, 12): 131551.2, (27, 10): 1601.175, (28, 10): 5916.55, (29, 14): 15549.9375, (30, 10): 4440.975, (31, 10): 2398.9, (32, 10): 14693.25, (33, 2): 210864.0, (34, 10): 6353.75, (35, 10): 5984.1, (36, 5): 64446.3875, (37, 10): 20525.575, (38, 7): 41851.125, (39, 10): 5658.0, (40, 10): 10086.0, (41, 10): 16196.5, (42, 10): 11550.0, (43, 10): 14550.0, (44, 12): 97476.15, (45, 10): 42605.625, (46, 10): 11183.25, (47, 10): 2174.375, (48, 10): 23899.8, (49, 10): 4001.55}
[1, 2, 3, 5, 7, 8, 9, 10, 12, 14]
67500
1072178.3875
10
T

<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [9]:
#Get the lowest shipping cost for each Client

start = time.time()
#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

{(0, 1): 17855.05, (1, 1): 12957.075, (2, 12): 64972.8, (3, 12): 142670.7, (4, 14): 10012.9375, (5, 12): 52974.65, (6, 12): 267015.0, (7, 12): 113296.35, (8, 14): 10369.7625, (9, 14): 9814.8, (10, 12): 469492.125, (11, 12): 86735.6, (12, 12): 142775.15, (13, 12): 20634.55, (14, 12): 47413.5, (15, 14): 47367.75, (16, 12): 24958.5, (17, 12): 281579.0, (18, 12): 29289.625, (19, 1): 21613.125, (20, 12): 10566.6, (21, 12): 63525.975, (22, 12): 54445.2, (23, 12): 35726.4, (24, 12): 66759.2, (25, 12): 36979.075, (26, 1): 414270.0, (27, 12): 57150.85, (28, 12): 44589.9, (29, 12): 57990.0, (30, 12): 26222.55, (31, 12): 34201.85, (32, 12): 60621.75, (33, 12): 1369070.4, (34, 12): 28836.25, (35, 12): 34446.75, (36, 12): 386255.425, (37, 12): 216573.175, (38, 14): 80899.3125, (39, 12): 40144.2, (40, 12): 140635.2, (41, 12): 99457.025, (42, 14): 30603.4375, (43, 1): 40075.0, (44, 14): 231908.1375, (45, 14): 83466.2875, (46, 14): 28787.025, (47, 1): 11428.575, (48, 12): 155437.2, (49, 12): 22479.45}

In [10]:
#customers = [(0,1.5), (2.5,1.2)]
#facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
print (num_facilities)
print (num_customers)

#print ("<><><><><><><><><><><><><><>")
#print (demand)
#print (alloc_cost)
print (fixed_cost)

print ("<><><><><><><><><><><><><><>")
# shipping costs
#print (shipping_cost)

16
50
[7500, 7500, 7500, 7500, 7500, 7500, 7500, 7500, 7500, 7500, 0, 7500, 7500, 7500, 7500, 7500]
<><><><><><><><><><><><><><>
